# Uebung 03

Gruppe: Josef Koch, Thomas Wally

## Anmerkungen

Für diese Übung ist es zugelassen, dass Sie in __Gruppen von zwei Personen__ arbeiten.
Sie dürfen auch alleine arbeiten wenn Sie das bevorzugen.
Beide Gruppenmitglieder müssen die Kreuzerlliste ausfüllen und die Ausarbeitung hochladen.
Auch müssen die Namen beider Gruppenmitglieder in der Ausarbeitung vermerkt werden.

Geben Sie bitte Quellen an wenn Sie Code aus dem Internet Kopieren.
Laden Sie Ihre Lösung nur in privaten Repos auf Platformen wie GitHub, GitLab, usw. hoch.

## Decision Tree und Random Forest

In dieser Uebung sollen Sie einen Decision Tree mit Hilfe des ID3-Algorithmus selbst implementieren.
Die Performance ihrer Implementierung sollen Sie mit der Implementierung einer Bibliothek vergleichen.
Im Anschluss sollen Sie die Performance des Decision Trees mit der des Random Forest vergleichen.
Zur Klassifizierung verwenden wir dieses Mal den [Breast Cancer Datensatz](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer).
Die unten angegebenen Files `breast-cancer_train.data` und `breast-cancer_test.data` finden Sie im Moodle.

### Aufgabe 03.01 (15 Punkte)

Implementieren Sie selbststaendig einen Decision Tree.
Verwenden Sie dazu den ID3-Algorithmus.

- [ ] Daten einlesen
- [ ] Decision Tree implementieren
- [ ] Decision Tree erstellen (trainieren mit `breast-cancer_train.data`)
- [ ] Accuracy mit `breast-cancer_test.data` berechnen
- [ ] Laufzeit des Training des Decision Tree angeben

Sie muessen kein Pruning fuer den Baum implementieren.
Sie muessen fuer diese Aufgabe keine _k_-Fold-Cross-Validation machen.

Implementierungshinweise:
* Fuer die Berechnung des Information Gain duerfen Sie Funktionen aus der `math` oder `numpy` Bibliotek verwenden.
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie duerfen fuer den Decision Tree __keine__ Machine Learning Bibliothek verwenden (ausgenommen der oben angefuehrten).

### Aufgabe 03.02 (5 Punkte)

Verwenden Sie eine fertige Implementierung des Decision Trees von __scikit-learn__(`sklearn`).

- [ ] Daten einlesen
- [ ] Decision Tree mit __scikit-learn__(`sklearn`) erstellen (trainieren mit `breast-cancer_train.data`)
- [ ] Accuracy mit `breast-cancer_test.data` berechnen
- [ ] Verlgeichen Sie die Accuracy Ihrer Implementierung der des Decision Tree den Sie mit __scikit-learn__ erstellt haben. Versuchen Sie den Unterschied in der Accuracy zu erklaeren.

Implementierungshinweise:
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie sollen __scikit-learn__(`sklearn`) fuer den Decision Tree verwenden.

### Aufgabe 03.03 (5 Bonuspunkte)

Verwenden Sie eine fertige Implementierung des Random Forest von __scikit-learn__(`sklearn`).

- [ ] Daten einlesen
- [ ] Random Forest mit __scikit-learn__(`sklearn`) erstellen. Tunen Sie die Hyperparameter mit _k_-Fold-Cross-Validation mit `breast-cancer_train.data`.
- [ ] Accuracy mit `breast-cancer_test.data` berechnen
- [ ] Verlgeichen Sie die Accuracy des Decision Tree den Sie mit erstellt haben __scikit-learn__ mit der des Random Forest den Sie mit __scikit-learn__ erstellt haben. Versuchen Sie den Unterschied in der Accuracy zu erklaeren.

Implementierungshinweise:
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie duerfen __scikit-learn__(`sklearn`) fuer _k_-Fold-Cross-Validation verwenden.
* Sie sollen __scikit-learn__(`sklearn`) fuer den Random Forest verwenden.

### Anmerkungen

Vergessen Sie nicht ihre Abgabe entsprechend zu dokumentieren.

Laden Sie die von Ihnen bearbeitete `.ipynb` Datei sowie ein dazugehoeriges `requirements.txt` File in einer `.zip` Datei im Moodle Kurs hoch.
Achten Sie darauf, dass keine Umlaute im Namen ihrer abgegebenen Files enthalten sind.

---
#### Data Features
1. Class: no-recurrence-events, recurrence-events
2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
3. menopause: lt40, ge40, premeno.
4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
6. node-caps: yes, no.
7. deg-malig: 1, 2, 3.
8. breast: left, right.
9. breast-quad: left-up, left-low, right-up, right-low, central.
10. irradiat: yes, no.

### Aufgabe 03.01

In [36]:
import pandas as pd
import math
from sklearn.metrics import accuracy_score
import timeit
import pprint



# Calculate the entropy of a given data frame column
# Lecture03.pdf, page 34
def calc_entropy(col):
    values = col.unique()
    ent = 0
    for value in values:
        # Amount of a specific value / total amount of values
        pi = col.value_counts()[value] / len(col)
        ent += - pi * math.log2(pi)
    return ent


# Calculate the information gain of a given data frame and attribute
# Lecture03.pdf, page 36
def calc_gain(df, attribute, target_attribute = 'class'):
    total_ent = calc_entropy(df[target_attribute])
    attribute_values = df[attribute].unique()
    attribute_ent = 0
    for value in attribute_values:
        value_df = df.loc[df[attribute] == value]
        entropy_df = calc_entropy(value_df[target_attribute])
        value_ratio = len(value_df) / len(df)
        attribute_ent += value_ratio * entropy_df
    return total_ent - attribute_ent


# Build the tree using the ID3 algorithm
# https://en.wikipedia.org/wiki/ID3_algorithm
def id3(df, attributes, target_attribute = 'class'):
    
    # If there is only one value of the target attribute in the data frame, then return this value
    if len(df[target_attribute].unique()) == 1:
        return df[target_attribute].unique()[0]
    
    # If the attributes are empty, then return most common value of the target attribute in the data frame
    if len(attributes) == 0:
        return df[target_attribute].mode()[0]
    
    # Otherwise start the ID3 algorithm
    # Get the attribute with the highest information gain
    gains = {}
    for attribute in attributes:
        gains[attribute] = calc_gain(df, attribute)
    best_attribute = max(gains, key=gains.get)
    
    # Build the tree
    tree = { best_attribute: {} }
    
    # Remove the best attribute from the attributes array
    attributes = [i for i in attributes if i != best_attribute]  
    # TODO: Der befehl drunter sollt des gleiche machen wie der oben
    # aber der tree is dann viel simpler und hat an besseren accuracy_score ??? 5/5 whys
    # attributes.remove(best_attribute)
    
    # Build subtree for each attribute value
    attribute_values = df[best_attribute].unique()
    for att_value in attribute_values:
        att_df = df.loc[df[best_attribute] == att_value]
        
        # If the new data frame is empty,
        # the subtree is simply the most common value of the target attribute in the data frame
        if len(att_df) == 0:
            subtree = df[target_attribute].mode()[0]
        else:
            # Recursively call the function again with the new data frame and updated attributes
            subtree = id3(att_df, attributes)
        
        # Now add the subtree to the above created tree
        tree[best_attribute][att_value] = subtree
    
    return tree


# Prediction based on a given query {"attribute_name_1": "attribute_value", ...}, tree and default value
# https://www.python-course.eu/Decision_Trees.php
def predict(query, tree, default):
    # Iterate through every attribute name of the query
    for key in list(query.keys()):
        # If the attribute name is in the list of values of the root node
        if key in list(tree.keys()):
            # Check if key and value from the query exist on the tree, return the default if not
            try:
                tree[key][query[key]] 
            except KeyError:
                return default
            
            result = tree[key][query[key]]
            
            # if the result is another tree (subtree) we call the function again (recursive)
            if isinstance(result,dict):
                return predict(query, result, default)
            
            else:
                return result
            
       
    
breast_features = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']
breast_train = pd.read_csv('breast-cancer_train.data', names=breast_features)
breast_test = pd.read_csv('breast-cancer_test.data', names=breast_features)

start = timeit.default_timer()

# Calculate the tree as a dictionary with the ID3 algorithm by providing the data frame and features without 'class'
breast_train_tree = id3(breast_train, breast_features[1:])

end = timeit.default_timer()

# The most common value of the target attribute in the data frame
breast_test_default = breast_test['class'].mode()[0]

# Create a dictionary from the breast test data without the 'class' column
breast_test_dict = breast_test.iloc[:, 1:].to_dict(orient = "records")

y_true = breast_test['class'].tolist()
y_pred = []

for breast_query in breast_test_dict:
    y_pred.append(predict(breast_query, breast_train_tree, breast_test_default))

print('Accuracy = %.2f%%' % (accuracy_score(y_true, y_pred)*100))
print('Time = %.2fs' % (end - start))
#pprint.pprint(breast_train_tree)




Accuracy = 66.23%
Time = 1.89s


---
### Aufgabe 03.02

---
### Aufgabe 03.03
